In [1]:
import os
import numpy as np
import pandas as pd
import warnings

from google.colab import drive

warnings.filterwarnings('ignore')
drive.mount("/content/drive")

os.chdir("drive/MyDrive/competition/2022-AI-competition-Round1") # Local Path
os.listdir()

Mounted at /content/drive


['competition_data',
 'submission',
 'playground.ipynb',
 'AutoML Baseline.ipynb',
 'Evaluator Module.ipynb',
 '2022-08-04 feature EDA.ipynb',
 "2022-08-05 LGB_train(hyeonbin's parameter tuning).ipynb",
 'Optuna Optimization.ipynb',
 '2022-08-06 Regressor model compare.ipynb',
 'catboost_info',
 'evaluator',
 '.git',
 '.gitignore',
 'README.md',
 'Updated CLF.ipynb',
 '2022-08-17 Evaluator Optuna (v0.4).ipynb',
 '2022-08-17 Ensemble Module.ipynb',
 '2022-08-11 Models(XGB_ET) Optimization.ipynb',
 '2022-08-06 Optimize ET.ipynb',
 'feature_EDA_correlation.ipynb',
 'Untitled0.ipynb',
 'model compare.ipynb',
 'GitHub Connection.ipynb',
 '2022-08-24 RF optimization.ipynb']

In [2]:
!pip install -r evaluator/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 
     |████████████████████████████████| 308 kB 71.3 MB/s 
     |████████████████████████████████| 81 kB 9.6 MB/s 
     |████████████████████████████████| 209 kB 81.4 MB/s 
     |████████████████████████████████| 78 kB 5.7 MB/s 
     |████████████████████████████████| 147 kB 45.2 MB/s 
     |████████████████████████████████| 112 kB 63.2 MB/s 
     |████████████████████████████████| 49 kB 7.4 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=fd3806460b65bf3c3371aba3622db3704b099be3cb06efc2a9c068d9f3ecf8fc
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [3]:
from evaluator.evaluator import Evaluator, Model
from evaluator.ensemble import EnsembleModel

train_df = pd.read_csv('competition_data/train.csv')
test_df = pd.read_csv("competition_data/test.csv")
submission_df = pd.read_csv("competition_data/sample_submission.csv")

In [4]:
evaluator = Evaluator(
    **Model(train_df, "rf", "clf").get_model()
)
evaluator.run()

,accuracy,precision,recall,f1-score,roc_auc,mae,train_acc
fold,,,,,,,
1,0.788059,0.792977,0.842428,0.816955,0.869988,0.332585,NaN
2,0.789309,0.778821,0.856569,0.815847,0.871158,0.336680,NaN
3,0.776180,0.773265,0.845419,0.807734,0.862145,0.340331,NaN
4,0.792748,0.805269,0.844986,0.824650,0.866543,0.334717,NaN
mean,0.786574,0.787583,0.847350,0.816296,0.867459,0.336078,1.0


In [8]:
params = {
	'n_estimators': 292, 
	'max_depth': 45, 
	'min_samples_split': 2, 
	'min_samples_leaf': 1, 
	'max_features': 0.729627795893577
}

In [9]:
evaluator = Evaluator(
    **Model(train_df, "rf", "clf", **params).get_model()
)
evaluator.run()

,accuracy,precision,recall,f1-score,roc_auc,mae,train_acc
fold,,,,,,,
1,0.782745,0.793600,0.828508,0.810678,0.866018,0.325727,NaN
2,0.794311,0.785677,0.855995,0.819330,0.869546,0.328896,NaN
3,0.780556,0.784469,0.834739,0.808824,0.860066,0.333047,NaN
4,0.780244,0.800210,0.824932,0.812383,0.860597,0.329595,NaN
mean,0.784464,0.790989,0.836044,0.812804,0.864057,0.329316,1.0


## Optimization

In [10]:
model = Model(train_df, "rf", "clf")

In [ ]:
initial_params = (
    ("n_estimators", "int", (200, 500)),
    ("max_depth", "int", (10, 70)),
    ("min_samples_split", "int", (2, 5)),
    ("min_samples_leaf", "static", 1),
    ("max_features", "float", (0.6, 0.8)),
    # ("min_weight_fraction_leaf", "float", (0.0, 0.2)),
    ("max_leaf_nodes", "int", (500, 2000)),
)
model.optimize(initial_params, n_runs=2)

[I 2022-08-24 06:29:40,478] A new study created in memory with name: no-name-d267adf4-1bc8-4e07-a0e6-984032bd46e8
[I 2022-08-24 06:31:25,887] Trial 0 finished with value: 0.8619900612792342 and parameters: {'n_estimators': 270, 'max_depth': 32, 'min_samples_split': 4, 'max_features': 0.6267218666784835, 'max_leaf_nodes': 1367}. Best is trial 0 with value: 0.8619900612792342.
[I 2022-08-24 06:33:54,858] Trial 1 finished with value: 0.8626407366911069 and parameters: {'n_estimators': 318, 'max_depth': 53, 'min_samples_split': 2, 'max_features': 0.7391789007754805, 'max_leaf_nodes': 1560}. Best is trial 1 with value: 0.8626407366911069.
[I 2022-08-24 06:37:11,718] Trial 2 finished with value: 0.8641640839633844 and parameters: {'n_estimators': 489, 'max_depth': 44, 'min_samples_split': 3, 'max_features': 0.6457296752130615, 'max_leaf_nodes': 1974}. Best is trial 2 with value: 0.8641640839633844.


In [ ]:
evaluator = Evaluator(
    **model.get_model()
)
evaluator.run()